In [1]:
import os
import librosa
import librosa.display
import IPython.display as ipd
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from matplotlib.gridspec import GridSpec

# Introdução
Neste notebook, será realizada uma análise detalhada sobre as características (features) de um sinal de voz que representa a fala da vogal /a/ em duas frequências - grave e aguda. O sinal será estudado tanto no domínio do tempo quanto da frequência.

## 1. Carregando os arquivos de audio

In [4]:
audio_file_grave = '.\Audio\grave.wav'
audio_file_agudo = '.\Audio\agudo.wav'

In [5]:
y_grave, sr = librosa.load(path=audio_file_grave, sr=None)

In [6]:
y_agudo, sr = librosa.load(path=audio_file_agudo, sr=None)

C:\Users\gabri\AppData\Local\Temp\ipykernel_5484\3205874221.py:1: UserWarning: PySoundFile failed. Trying audioread instead.
  y_agudo, _ = librosa.load(path=audio_file_agudo, sr=None)


OSError: [Errno 22] Invalid argument: '.\\Audio\x07gudo.wav'

In [ ]:
# Reprodutor
ipd.Audio(audio_file)

## Visualização da forma de onda $y(t)$

In [ ]:
print(f'Número de amostras            : {y.size}')
print(f'Frequência de amostragem (sr) : {sr} amostras/segundo')
print(f'Duração do sinal (audio)      : {round(y.size/sr,2)} segundos')

In [ ]:
# Obtendo os instantes de tempo a partir da frequencia de amostragem
t = np.linspace(0,y.size/sr,y.size)

Exibe a forma de onda $y(t)$

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(t,y,alpha=0.5,lw=1)
plt.xlim(0,max(t))

# Removendo a região de silêncio
É possível notar que há regiões onde a amplitude do sinal é muito pequena. Para essas faixas, iremos remover as amostras do seguinte modo:
* Definir limiar mínimo de amplitude;
* Filtrar as amostras que satisfazem (ou não satisfazem) a condição de amplitude mínima;
* Obter array resultante somente com as amostras de interesse.

In [ ]:
amp_min = 0.1

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(t,y,alpha=0.5,lw=1)
plt.axhline(amp_min,label='$A_{min}$',color='tab:red')
plt.legend()
plt.xlim(0,max(t))

In [ ]:
y_trimmed = y[abs(y)>amp_min]
t_trimmed = t[abs(y)>amp_min]
t_trimmed = t_trimmed - min(t_trimmed)

In [ ]:
plt.figure(figsize=(18,4))
plt.plot(t_trimmed,y_trimmed,alpha=0.5,lw=1)

# Analisando as características no domínio do tempo mais de perto
Na escala acima,é difícil notar as características presentes na forma de onda. Vamos dar um zoom sobre determinada região e inspecionar o sinal e avaliar se existem regularidades - uma vez que trata-te de um sinal de voz de uma vogal.

In [ ]:
t_max,t_min = max(t_trimmed), min(t_trimmed)
regiao_zoom = [0.35,0.40]

In [ ]:
fig = plt.figure(figsize=(20,6))
# ,sharey=True,gridspec_kw={'width_ratios':[2,1],'wspace':0.05}
gs = GridSpec(nrows=2,ncols=2,figure=fig,width_ratios=(3,1),height_ratios=(1,2),hspace=(0.15),wspace=(0.05))

ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1],sharey=ax1)
ax3 = fig.add_subplot(gs[1,:])

ax1.plot(t_trimmed,y_trimmed,alpha=0.5,lw=1)
ax1.set_xlim(t_min,t_max)
ax1.set_title('\ a \ em toda sua duração',loc='left')

ax2.plot(t_trimmed,y_trimmed,alpha=0.5,lw=1)
ax2.set_title('\ a \ em uma janela de 0.05s',loc='left')
ax2.set_yticks([])
ax2.set_xlim(regiao_zoom)

ax3.plot(t_trimmed,y_trimmed,alpha=0.5,lw=1)
ax3.set_xlim(0.355,0.370)
plt.show()

> Do gráfico acima, é possível perceber uma repetição num padrão de onda. Esse padrão fora destacado na figura acima com período de aproximadamente 0.01s

## Transformada de Fourier de $y(t)$

In [ ]:
N = y.size      # Número de amostras
T = 1.0/sr      # periodo de amostragem (inverso da frequencia de amostragem)

In [ ]:
y_fourier = sp.fft.fft(y)

f_fourier = sp.fft.fftfreq(N, T)

Análise da magnitude de $\hat{y}(f)$

In [ ]:
# Magnitude (módulo) e fase da transformada
mag = abs(y_fourier)

In [ ]:
print(f'Média  : {np.mean(mag)}')
print(f'Mediana: {np.median(mag)}')
print(f'Desvio padrão: {np.std(mag)}')

In [ ]:
plt.figure(figsize=(16,4))
ax = plt.subplot()

# Magnitude
ax.plot(f_fourier[:N//2],mag[:N//2],color='tab:blue',alpha=0.5)
ax.set_ylabel(r'$|\hat{y}(\omega)|$')
ax.set_xlabel('Frequência (Hz)')
ax.set_xlim(0,)
ax.set_ylim(0,)

In [ ]:
fig = plt.figure(figsize=(20,6))
gs = GridSpec(nrows=1,ncols=2,figure=fig,width_ratios=(1,3),wspace=(0.05))

ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1],sharey=ax1)

ax1.plot(f_fourier[:N//2],mag[:N//2],alpha=0.5,lw=1)
ax1.set_xlim(0,)
ax1.set_ylim(0,)
ax1.set_xlabel('Frequência (Hz)')


ax2.plot(f_fourier[:N//2],mag[:N//2],alpha=0.5,lw=1)
ax2.set_xlim(0,5e3)
ax2.yaxis.set_tick_params(labelleft=False)
ax2.set_xlabel('Frequência (Hz)')
plt.show()

In [ ]:
from scipy.signal import spectrogram

In [ ]:
sr1 = (t_trimmed.size)/max(t_trimmed)

In [ ]:
f, t, Sxx = spectrogram(y_trimmed, sr1)
Sxx[Sxx == 0] = 1e-10

In [ ]:
# Plot the spectrogram
plt.figure(figsize=(16, 3))
plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud',cmap='inferno')
plt.ylabel('Frequencia (Hz)')
plt.xlabel('Tempo (s)')
plt.title('Espectrograma')
plt.colorbar(label='Intensidade [dB]',pad=0.01)
plt.show()

## Transformada de Fourier somente na janela

In [ ]:
t_janela = t_trimmed[(t_trimmed >= 0.355) & (t_trimmed <= 0.370)]
y_janela = y_trimmed[(t_trimmed >= 0.355) & (t_trimmed <= 0.370)]

In [ ]:
sr_w = (t_janela.size)/max(t_janela)

In [ ]:
N_janela = y_janela.size  # Número de amostras
T_janela = 1.0/sr_w       # periodo de amostragem (inverso da frequencia de amostragem)

In [ ]:
y_fourier_janela = sp.fft.fft(y_janela)
f_fourier_janela = sp.fft.fftfreq(N_janela, T_janela)

In [ ]:
f_janela, t_janela, Sxx_janela = spectrogram(y_janela, sr_w)
# Sxx_janela[Sxx_janela == 0] = 1e-10

In [ ]:
# Create the figure and GridSpec layout
fig = plt.figure(figsize=(20, 12))  # Increased the height to accommodate two rows
gs = GridSpec(nrows=2, ncols=2, figure=fig, width_ratios=(1, 3), height_ratios=(1, 1), wspace=0.05, hspace=0.3)

# First row, first plot
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(f_fourier[:N//2], mag[:N//2], alpha=0.5, lw=1)
ax1.set_xlim(0,)
ax1.set_ylim(0,)
ax1.set_xlabel('Frequência (Hz)')
ax1.set_title('First Row, First Plot')

# First row, second plot
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(f_fourier_janela[:N_janela//2], abs(y_fourier_janela)[:N_janela//2], alpha=0.5, lw=1)
ax2.yaxis.set_tick_params(labelleft=False)
ax2.set_ylim(0,)
ax2.set_xlim(0,)
ax2.set_xlabel('Frequência (Hz)')
ax2.set_title('First Row, Second Plot')

# Second row, first plot (spectrogram)
ax3 = fig.add_subplot(gs[1, 0])
pcm1 = ax3.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud', cmap='inferno')
ax3.set_ylabel('Frequencia (Hz)')
ax3.set_xlabel('Tempo (s)')
ax3.set_title('Second Row, First Plot (Spectrogram)')
fig.colorbar(pcm1, ax=ax3, label='Intensidade [dB]', pad=0.01)

# Second row, second plot (spectrogram)
ax4 = fig.add_subplot(gs[1, 1])
pcm2 = ax4.pcolormesh(t_janela, f_janela, 10 * np.log10(Sxx_janela), shading='gouraud', cmap='inferno')
ax4.set_xlabel('Tempo (s)')
ax4.set_title('Second Row, Second Plot (Spectrogram)')
ax4.yaxis.set_tick_params(labelleft=False)
fig.colorbar(pcm2, ax=ax4, label='Intensidade [dB]', pad=0.01)

plt.show()